# Data Analysis, Linear Regression and Model Comparison 

In this notebook we are going to consider historical measurements of water quality data coming from a surface water station in Berlin.

First, we need to import the necessary libraries to load and handle data.

In [208]:
import os
import pandas as pd

In [209]:
# Set the data path
data_path = "data"

Let's use pandas to load the data into a **pandas.DataFrame**.
Since we received three csv files, one for measurements regarding water quality parameters, one for flow rate measurements and one for rain precipitations, we are going to load three csv files in three separate pandas.DataFrame that we are going to merge later.

In [210]:
ts_sw_df = pd.read_csv(
    os.path.join(
        data_path, "time-series_surface-water_quality.csv"
    )
)
flow_df = pd.read_csv(
    os.path.join(
        data_path, "time-series_surface-water_flow.csv"
    )
)

Let's have a first look at the datasets.

In [ ]:
ts_sw_df.head()

In [ ]:
flow_df.head()

In order to understand them, it could be helpful to translate them to english :)

In [213]:
ts_sw_df.rename(
    columns={
        "Messstelle": "Station",
        "Messstellennummer": "Station ID",
        "Datum": "DateTime",
        "Einheit": "Unit",
        "Wert": "Value",
    },
    inplace=True,
)

# Drop unnecessary columns
ts_sw_df.drop(
    columns=[
        "Entnahmetiefe [m]",
        "Messmethode",
        "Vorzeichen",
        "Bestimmungsgrenze",
    ],
    inplace=True,
)

In [214]:
# Define the translation dictionary for the variables we are interested in
variables_mapping = {
    "Lufttemperatur": "Air Temperature (°C)",
    "Wassertemperatur": "Water Temperature (°C)",
    "Leitfähigkeit": "Conductivity (µS/cm)",
    "Ammonium-Stickstoff": "Ammonium (mg/l)",
    "Nitrat-Stickstoff": "Nitrate (mg/l)",
    "pH-Wert": "pH",
    "DOC (Gelöster organischer Kohlenstoff)": "DOC (mg/l)",
    "Sauerstoff-Gehalt": "Dissolved Oxygen (mg/l)",
}
# Filter the data frame to only include the variables we are interested in
ts_sw_df = ts_sw_df[ts_sw_df["Parameter"].isin(variables_mapping.keys())]

# Rename the variables
ts_sw_df["Parameter"] = ts_sw_df["Parameter"].map(variables_mapping)

In [215]:
flow_df.rename(
    columns={
        "Messstellennummer": "Station ID",
        "Datum": "DateTime",
        "Tagesmittelwert": "Flow Rate (m³/s)",
    },
    inplace=True,
)

# Drop unnecessary columns
flow_df.drop(
    columns=[
        "Parameter",
        "Einheit",
    ],
    inplace=True,
)

For simplicity, we are going to work just on the data coming from station 325

In [216]:
ts_sw_df = ts_sw_df[ts_sw_df["Station ID"] == 305]

# close to the station 325
flow_df = flow_df[flow_df["Station ID"] == 5803200]

# Remove the Station ID column as it is not needed anymore
flow_df.drop(columns=["Station ID"], inplace=True)

In [ ]:
# Let's have a look at the DataFrame
ts_sw_df.head()

In [218]:
# In order to have a column for each variable, we are going to pivot the DataFrame
ts_sw_df = ts_sw_df.pivot(
    index="DateTime", columns="Parameter", values="Value"
)

In [ ]:
ts_sw_df.head()

We are ready to merge the water quality data with the flow data, using the DateTime as the key

In [220]:
# First, we need to convert the DateTime to a datetime object
ts_sw_df.index = pd.to_datetime(ts_sw_df.index)

flow_df["DateTime"] = pd.to_datetime(flow_df["DateTime"])
flow_df.set_index("DateTime", inplace=True)

# We are going to merge the two DataFrames on just the date (not the time)
ts_sw_df.index = ts_sw_df.index.date
flow_df.index = flow_df.index.date

# Merge the two DataFrames with a left join since
# we want to keep all the water quality data even if there is no flow data
station_df = ts_sw_df.merge(
    flow_df, left_index=True, right_index=True, how='left'
)

# Recast the index to a datetime object
station_df.index = pd.to_datetime(station_df.index)

In [ ]:
# Let's have a look at the DataFrame
station_df.head()

Let's gather some information regarding the data.

In [222]:
info_station_df = pd.DataFrame(
    index=pd.Index(
        [
            "N Samples",
            "% Missing Values",
            "Frequency (days)",
            "Mean",
            "Std",
            "Start Date",
            "End Date",
        ],
        name="Info",
    ),
    columns=station_df.columns,
)

In [223]:
# store the information in the station_info_df
for column in station_df.columns:
    
    # Get the column of interest
    df = station_df[column].copy()

    # Get the start and end date of the time series,
    # from the first non missing value to the last one
    start_date = df.dropna().index.min()
    end_date = df.dropna().index.max()

    # Filter the DataFrame to only include the period of interest
    df = df[start_date:end_date]

    # Calculate the percentage of missing values in the period of interest
    missing_values = df.isna().sum() / df.shape[0] * 100

    # Store the number of valid samples
    info_station_df.loc["N Samples", column] = (
        station_df[column].dropna().shape[0]
    )
    # Store the percentage of missing values
    info_station_df.loc[
        "% Missing Values", column
    ] = missing_values.round(3)
    
    # Store the most common frequency of the time series
    info_station_df.loc["Frequency (days)", column] = (
        station_df.index.to_series().diff().value_counts().index[0].days
    )
    
    # Store the mean and standard deviation of the time series
    info_station_df.loc["Mean", column] = df.mean().round(3)
    info_station_df.loc["Std", column] = df.std().round(3)
    
    # Store the start and end date of the time series
    info_station_df.loc["Start Date", column] = start_date.strftime("%Y-%m-%d")
    info_station_df.loc["End Date", column] = end_date.strftime("%Y-%m-%d")

In [ ]:
info_station_df

# Preprocessing

## Water Quality + Flow Rate

From the summary table we can see that the % of missing values is quite small for each parameter, and the most common sampling frequency is monthly. Since we need to have a common time interval for every variable, we are going to cut each time series from the latest Start Date (Flow Rate) to the earliest End Date (Ammonium).

In [225]:
start_date = station_df["Flow Rate (m³/s)"].dropna().index.min()
end_date = station_df["Ammonium (mg/l)"].dropna().index.max()

station_df = station_df[start_date:end_date]

Let's start to plot the variables. We are going to use matplotlib library.

In [226]:
import matplotlib.pyplot as plt

In [ ]:
for column in station_df.columns:
    
    df = station_df[column].copy()
    
    plt.figure(figsize=(12, 6))
    
    # Plot the time series
    plt.plot(station_df.index, station_df[column], label=column)
    
    # Add a vertical line for each date with missing values
    missing_dates = station_df[station_df[column].isna()].index
    for date in missing_dates:
        plt.axvline(date, color="red", linestyle="--", alpha=0.5)
        
    # add legend for missing values line if there are any
    if len(missing_dates) > 0:
        plt.plot([], color="red", linestyle="--", alpha=0.5, label="Missing Values")
        
    # Add a title
    plt.title(column)
    
    plt.grid(True)
    
    # Add a legend
    plt.legend()
    
    plt.ylabel(column)
    plt.xlabel("Date")
    
    plt.show()

From the graphs we see that the data is quite clean. For the outliers' removal process we are going to be more conservative as water quality data has strong seasonal effects and could be noisy.

We are going to use [Prophet](https://facebook.github.io/prophet/) to detect outliers. What we are going to do is:
- compute the residuals of the Prophet predictions:
$$
r(t) = y(t) - \hat{y}(t)
$$

- define an anomaly if:
$$
|r(t)| > f * (\hat{y}_{hb}(t) - \hat{y}_{lb}(t))
$$

where *f* is an anomaly factor that can be tuned based on your choice. An outlier is identified as such if the residual of its prediction is greater than *f* times the difference between the upper and lower prediction bounds provided by Prophet, indicating that the actual value significantly deviates from the expected range.

In [228]:
# import the necessary libraries
from prophet import Prophet

import numpy as np

In [ ]:
factor = 3 # to be more conservative, usually 1.5 to 3

for column in station_df.columns:
    df = station_df[column].copy()
    
    fig, axs = plt.subplots(2, 1, figsize=(12, 12))

    # ===== Prophet =====

    df.index.name = "ds"

    df = df.reset_index()

    df.rename(columns={column: "y"}, inplace=True)

    # using prophet
    model = Prophet()
    model.fit(df)
    
    # We don't want to predict anything, just get the forecast for the training data
    future = model.make_future_dataframe(periods=0)
    forecast = model.predict(future)

    # Merging forecasted data with original data
    forecasting_final = pd.merge(
        forecast[["ds", "yhat", "yhat_lower", "yhat_upper"]],
        df,
        how="inner",
        on="ds",
    )

    # Calculate the prediction error (residual) and uncertainty
    forecasting_final["error"] = (
        forecasting_final["y"] - forecasting_final["yhat"]
    )
    forecasting_final["uncertainty"] = (
        forecasting_final["yhat_upper"]
        - forecasting_final["yhat_lower"]
    )

    # Anomaly detection
    
    # We are going to consider an anomaly if the error is greater than factor * uncertainty
    forecasting_final["anomaly"] = forecasting_final.apply(
        lambda x: "Yes"
        if (np.abs(x["error"]) > factor * x["uncertainty"])
        else "No",
        axis=1,
    )

    anomaly = forecasting_final[forecasting_final["anomaly"] == "Yes"]
    
    axs[0].plot(forecasting_final["ds"], forecasting_final["y"], label="Actual")
    axs[0].plot(forecasting_final["ds"], forecasting_final["yhat"], label="Prophet Prediction")
    
    if not anomaly.empty:
        axs[0].scatter(anomaly["ds"], anomaly["y"], label="Prophet Outliers", color="red")
    
    axs[1].plot(forecasting_final["ds"], forecasting_final["error"], label="Error")
    axs[1].plot(forecasting_final["ds"], forecasting_final["uncertainty"], label="Uncertainty")
    
    if not anomaly.empty:
        axs[1].scatter(anomaly["ds"], anomaly["error"], label="Prophet Outliers", color="red")

    
    axs[0].set_title(column)
    axs[0].set_xlabel("Date")
    axs[0].set_ylabel(column)
    axs[0].legend()
    axs[0].grid()
    
    axs[1].set_title("Error and Uncertainty")
    axs[1].set_xlabel("Date")
    axs[1].set_ylabel("Error and Uncertainty")
    axs[1].legend()
    axs[1].grid()
    
    plt.tight_layout()
    plt.show()

As we can see, just two points have been labeled as anomalies. We are going to remove them.

In [ ]:
# columns to be processed

for column in station_df.columns:
    df = station_df[column].copy()

    df.index.name = "ds"

    df = df.reset_index()

    df.rename(columns={column: "y"}, inplace=True)

    # using prophet
    model = Prophet()
    model.fit(df)
    
    # Make predictions for both columns
    future = model.make_future_dataframe(periods=0)
    forecast = model.predict(future)

    # Merging forecasted data with the original data
    forecasting_final = pd.merge(
        forecast[["ds", "yhat", "yhat_lower", "yhat_upper"]],
        df,
        how="inner",
        on="ds",
    )

    # Calculate the prediction error and uncertainty
    forecasting_final["error"] = (
        forecasting_final["y"] - forecasting_final["yhat"]
    )
    forecasting_final["uncertainty"] = (
        forecasting_final["yhat_upper"]
        - forecasting_final["yhat_lower"]
    )

    # Anomaly detection
    forecasting_final["anomaly"] = forecasting_final.apply(
        lambda x: "Yes"
        if (np.abs(x["error"]) > factor * x["uncertainty"])
        else "No",
        axis=1,
    )

    # remove the outliers by replacing them with NaN
    forecasting_final.loc[
        forecasting_final["anomaly"] == "Yes", "y"
    ] = np.nan

    df = forecasting_final[["ds", "y"]]

    df.set_index("ds", inplace=True)

    df = df.rename(columns={"y": column})

    station_df.loc[df.index, column] = df[column]


To make the analysis easier, we are going to resample the data to a monthly frequency
and calculate the mean value for each month if there are multiple samples in the same month.

Also, the resampling will ease the missing values imputation as we will have a regular time series.

In [231]:
station_df = station_df.resample("ME").mean()

# We are going to impute the missing values using the interpolation method for the time series
# This method will fill the missing values by interpolating between the two closest non missing values
station_df = station_df.interpolate(method="time")

In [ ]:
# Show the plots of preprocessed data, with missing values filled
for column in station_df.columns:
    
    df = station_df[column].copy()
    
    plt.figure(figsize=(12, 6))
    
    # Plot the time series
    plt.plot(station_df.index, station_df[column], label=column)
    
    # Add a vertical line for each date with missing values
    missing_dates = station_df[station_df[column].isna()].index
    for date in missing_dates:
        plt.axvline(date, color="red", linestyle="--", alpha=0.5)
        
    # add legend for missing values line if there are any
    if len(missing_dates) > 0:
        plt.plot([], color="red", linestyle="--", alpha=0.5, label="Missing Values")
        
    # Add a title
    plt.title(column)
    
    plt.grid(True)
    
    # Add a legend
    plt.legend()
    
    plt.ylabel(column)
    plt.xlabel("Date")
    
    plt.show()

In [ ]:
# Let's have a final description of the data
station_df.describe()

## Meteorological Data

As a final step to build the final dataset, we need to add the precipitation measurements available from an airport quite distant from the station. In order to assess similarity between the two sampling points, we have to evaluate the similarity of the measurements of common variables. The only variable in common between the two DataFrames is the Air Temperature.

In [234]:
# load the data
meteo_df = pd.read_csv(
    os.path.join(
        data_path, "produkt_klima_tag_19480101_20231231_00433.csv",
    ),
    sep=";",
)

In [235]:
meteo_df.rename(
    columns={
        "STATIONS_ID": "Station ID",
        "MESS_DATUM": "DateTime",
        "  FX": "Wind Speed Max (m/s)",
        "  FM": "Wind Speed Mean (m/s)",
        " RSK": "Cumulated Rainfall (mm)",
        "RSKF": "Cumulated Rainfall Type",
        " SDK": "Sunshine Duration (hours)",
        "SHK_TAG": "Snow Height (cm)",
        "  NM": "Cloud Coverage (1/8)",
        " VPM": "Vapor Pressure (hPa)",
        "  PM": "Pressure (hPa)",
        " TMK": "Temperature Mean (°C)",
        " UPM": "Humidity (%)",
        " TXK": "Temperature Max at 2m (°C)",
        " TNK": "Temperature Min at 2m (°C)",
        " TGK": "Temperature Min at 5cm (°C)",
    },
    inplace=True,
)

meteo_df["DateTime"] = pd.to_datetime(meteo_df["DateTime"], format="%Y%m%d")
meteo_df.set_index("DateTime", inplace=True)

In [236]:
# Just take the necessary columns
meteo_df = meteo_df[
    [
        "Temperature Mean (°C)",
        "Cumulated Rainfall (mm)",
    ]
]

In [237]:
# Inspect the DataFrame
info_meteo_df = pd.DataFrame(
    index=pd.Index(
        [
            "N Samples",
            "% Missing Values",
            "Frequency (days)",
            "Mean",
            "Std",
            "Start Date",
            "End Date",
        ],
        name="Info",
    ),
    columns=meteo_df.columns,
)

In [ ]:
for column in meteo_df.columns:
    start_date = meteo_df[column].dropna().index.min()
    
    end_date = meteo_df[column].dropna().index.max()

    df = meteo_df[start_date:end_date][column]

    print(f"Start date for {column}: {start_date}")
    print(f"End date for {column}: {end_date}")

    missing_values = df.isna().sum() / df.shape[0]
    print(f"Missing values for {column}: {missing_values}")

    frequency = df.index.to_series().diff().value_counts().index[0].days
    print(f"Frequency for {column}: {frequency}")

    info_meteo_df.loc["N Samples", column] = (
        meteo_df[column].dropna().shape[0]
    )
    info_meteo_df.loc["% Missing Values", column] = missing_values
    info_meteo_df.loc["Frequency (days)", column] = frequency
    
    info_meteo_df.loc["Mean", column] = df.mean()
    info_meteo_df.loc["Std", column] = df.std()
    
    info_meteo_df.loc["Start Date", column] = start_date.strftime("%Y-%m-%d")
    info_meteo_df.loc["End Date", column] = end_date.strftime("%Y-%m-%d")

In [ ]:
info_meteo_df

We can see that there are not missing values in the time series. The time range is very long, from 1948 to 2023. The sampling frequency is daily, so we need to adjust it to monthly.

However, the standard deviation of the Cumulated Rainfall is a bit high, let's visualize the plots.

In [ ]:
# Visually inspect the data

for column in meteo_df.columns:
    fig = plt.figure(figsize=(12, 6))

    plt.plot(meteo_df.index, meteo_df[column], label=column)

    plt.title(column)
    plt.xlabel("Date")
    plt.ylabel(column)
    plt.legend()

    plt.show()

The cumulated rainfall has some errors in the measurements, let's delete the measurements that are < 0.

In [241]:
meteo_df.loc[
    meteo_df["Cumulated Rainfall (mm)"] < 0, ["Cumulated Rainfall (mm)"]
] = np.nan

In [ ]:
column = "Cumulated Rainfall (mm)"

fig = plt.figure(figsize=(12, 6))

plt.plot(meteo_df.index, meteo_df[column], label=column)

plt.title(column)
plt.xlabel("Date")
plt.ylabel(column)
plt.legend()

plt.show()

The values are now reasonable. Let's resample to monthly measurements and fill the missing values.

In [243]:
# resample the data to monthly
meteo_df = meteo_df.resample("ME").mean()

meteo_df = meteo_df.interpolate(method="time")

We are now going to compare the air temperature measurements of the two sites to understand if they can be considered similar, in order to merge the rainfall measurements in the final dataset. We compute the pearson correlation as a proxy of how much the two time series are correlated. We also show visual evidence as well as a scatter plot.

In [244]:
from scipy.stats import pearsonr

In [ ]:
# compute pearson correlation

start_date = station_df.index.min()
end_date = station_df.index.max()

# take the common date range with the airport
start_date = max(start_date, meteo_df.index.min())
end_date = min(end_date, meteo_df.index.max())

airport_df = meteo_df[start_date:end_date].copy()

# take the common date range with the station
df = station_df[
    (station_df.index >= start_date)
    & (station_df.index <= end_date)
].copy()

# compute pearson correlation
corr, _ = pearsonr(
    airport_df["Temperature Mean (°C)"],
    df["Air Temperature (°C)"],
)

fig, ax = plt.subplots(figsize=(10, 8))

# Plot Airport data
ax.plot(airport_df.index, airport_df["Temperature Mean (°C)"], color="blue", label="Airport")

# Plot Station data
ax.plot(df.index, df["Air Temperature (°C)"], color='green', label=f"Surface Station")

# Add the correlation to the plot
ax.text(0.01, 0.95, f"Pearson Correlation: {corr:.2f}", transform=ax.transAxes, fontsize=18, verticalalignment='top')

# Set labels and title
ax.set_xlabel("Date")
ax.set_ylabel("Air Temperature (°C)")
ax.legend()
ax.set_title("Air Temperature")

plt.show()

In [ ]:
# The only variable in common between the two DataFrames is the Temperature Mean
start_date = station_df.index.min()
end_date = station_df.index.max()

# take the common date range with the airport
start_date = max(start_date, meteo_df.index.min())
end_date = min(end_date, meteo_df.index.max())

airport_df = meteo_df[start_date:end_date].copy()

# take the common date range with the station
df = station_df[
    (station_df.index >= start_date)
    & (station_df.index <= end_date)
].copy()

fig, ax = plt.subplots(figsize=(10, 8))

# Plot Data
ax.scatter(airport_df["Temperature Mean (°C)"], df["Air Temperature (°C)"], 
           s=64, color="blue", alpha=0.7, label="Data")

# Add line on bisector
ax.plot([-10, 40], [-10, 40], color="red", linewidth=2, linestyle="--", label="Bisector")

# Set labels and title
ax.set_xlabel("Airport")
ax.set_title("Air Temperature")
ax.legend(loc="upper right")

plt.show()

We have strong correlation between the two measurements, and visually we can see that the seasonal pattern is very similar. We can establish that they are similar, and we can merge the rainfall into the final dataset.

In [247]:
station_df['Cumulated Rainfall (mm)'] = np.nan

start_date = station_df.index.min()
end_date = station_df.index.max()

# take the common date range with the airport
start_date = max(start_date, meteo_df.index.min())
end_date = min(end_date, meteo_df.index.max())

airport_df = meteo_df[start_date:end_date].copy()

# take the common date range with the station
# Identify the indices in sw_df that match the station_id and are within the date range
station_df.loc[
    (station_df.index >= start_date)
    & (station_df.index <= end_date),
    "Cumulated Rainfall (mm)",
] = airport_df["Cumulated Rainfall (mm)"]

# interpolate the missing value of the Cumulated Rainfall
station_df["Cumulated Rainfall (mm)"] = station_df["Cumulated Rainfall (mm)"].interpolate()

In [ ]:
# Plot the data
fig, ax = plt.subplots(figsize=(10, 8))

# Plot Data
ax.plot(station_df.index, station_df["Cumulated Rainfall (mm)"], color="blue", label="Airport")

# Set labels and title
ax.set_xlabel("Date")
ax.set_ylabel("Cumulated Rainfall (mm)")

plt.show()


In [ ]:
# final overview of the data
station_df.describe()

In [250]:
# Let's save the final DataFrame for future notebooks
station_df.to_excel(
    os.path.join(data_path, "clean_data.xlsx")
)

# Linear Regression

We can now start performing linear regression on historical data to predict our target variable (DOC (mg/l)) with respect to the regressors. 

We are going to use the [statsmodels](https://www.statsmodels.org/stable/index.html) package as it provides more insights for linear regression tasks.

In [251]:
# import the necessary libraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import statsmodels.api as sm

First of all, we need to normalize our exogenous variables to ensure better convergence of OLS, since there is a high difference in the scale of their domains.


In [252]:
# split the dataframe into features and target
X = station_df.drop(columns=["DOC (mg/l)"]).copy()
y = station_df["DOC (mg/l)"].copy()

# split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=False
)

# scale the data
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)

# revert to dataframe
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X.columns, index=X_train.index)

In [253]:
# fit the model
X_train_scaled = sm.add_constant(X_train_scaled)

model = sm.OLS(y_train, X_train_scaled).fit()

In [254]:
X_test_scaled = scaler.transform(X_test)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X.columns, index=X_test.index)
X_test_scaled = sm.add_constant(X_test_scaled)

# make predictions
y_pred = model.predict(X_test_scaled)

To evaluate the quality of our regression we can analyse some metrics:

**Residual Sum of Squares**

$RSS = \sum_n (\hat{t}_n-t_n)^2$, it tells us how much of the prediction differs from the true value.

In [ ]:
RSS = np.sum((y_test - y_pred) ** 2)
RSS

**(Adjusted) Coefficient of determination**



$R^2_{\text{adj}} = 1 - \left( \frac{(1 - R^2)(n - 1)}{n - p - 1} \right)$, where $R^2 = 1 - \frac{RSS}{\sum_n (\bar{t}-t_n)^2}$. It is a modified version of $R^2$ that accounts for the number of predictors (independent variables) in the model. It penalizes the inclusion of unnecessary variables that do not improve the model significantly.

$R^2 = 1 - \frac{RSS}{\sum_n (\bar{t}-t_n)^2}$, it tells us how the fraction of the variance of the data explained by the model (how much better we are doing w.r.t. just using the mean of the target $\bar{t} = \frac{\sum_n t_n}{N}$).

In spaces with a single feature this is equal to the correlation coefficient between the input and the output;

For a more detailed explanation: https://en.wikipedia.org/wiki/Coefficient_of_determination

Statsmodels offers a summary with a lot of information, including $R^2$ and $R^2_{\text{adj}}$.

In [ ]:
print(model.summary())

**Mean Squared Error**

$MSE = \frac{\sum_n (\hat{t}_n-t_n)^2}{N}$, it tells approximately how much error we get on a predicted data over the training set (i.e., a normalized version of the RSS).

In [257]:
from sklearn.metrics import mean_squared_error

In [ ]:
mean_squared_error(y_test, y_pred)

We can see how the standard linear regression doesn't perform quite well since it makes the strong assumption of linear dependencies between the target variable and the features, which in real-world problems it is usually not the case.

More complex models able to capture non-linear relationships are often used. However, understanding which model is best suited for a specific problem is not straightforward. The process of model selection involves different tasks, from feature selection to hyperparameter tuning. Here a first naive comparison between different models is performed.

The models we are going to compare are:
- Linear Regression
- Random Forest
- Multi-layer Perceptron Neural Network
- XGBoost

Since Random Forest, MLP and XGBoost are non-deterministic types of models, to evaluate them we are going to use cross-validation to reduce the impact of randomness of these models.

# Model Comparison

In [259]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import cross_validate

from xgboost import XGBRegressor

In [260]:
# save the linear regression results
y_pred_lr = y_pred.copy()    

In [261]:
# split the dataframe into features and target
X = station_df.drop(columns=["DOC (mg/l)"]).copy()
y = station_df["DOC (mg/l)"].copy()

# split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=False
)

# scale the data
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)

# revert to dataframe
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X.columns, index=X_train.index)

In [ ]:
# perform cross validation for the different models

# Random Forest
rf_cv = cross_validate(
    RandomForestRegressor(),
    X_train_scaled,
    y_train,
    cv=5, # 5-fold cross validation
    scoring="neg_mean_squared_error", # we want to minimize the mean squared error
    return_estimator=True, # to get the fitted models for final evaluation on the test set
)

# Neural Network
nn_cv = cross_validate(
    MLPRegressor(max_iter=1000),
    X_train_scaled,
    y_train,
    cv=5, # 5-fold cross validation
    scoring="neg_mean_squared_error", # we want to minimize the mean squared error
    return_estimator=True, # to get the fitted models for final evaluation on the test set
)

# XGBoost
xgb_cv = cross_validate(
    XGBRegressor(),
    X_train_scaled,
    y_train,
    cv=5, # 5-fold cross validation
    scoring="neg_mean_squared_error", # we want to minimize the mean squared error
    return_estimator=True, # to get the fitted models for final evaluation on the test set
)

In [273]:
# predict on the test set

# Random Forest
rf_estimators = rf_cv["estimator"]
rf_predictions = np.zeros((X_test.shape[0], len(rf_estimators)))

for i, rf in enumerate(rf_estimators):
    X_test_scaled = scaler.transform(X_test)
    X_test_scaled = pd.DataFrame(X_test_scaled, columns=X.columns, index=X_test.index)
    rf_predictions[:, i] = rf.predict(X_test_scaled)
    
y_pred_rf = np.mean(rf_predictions, axis=1)

# Neural Network
nn_estimators = nn_cv["estimator"]
nn_predictions = np.zeros((X_test.shape[0], len(nn_estimators)))

for i, nn in enumerate(nn_estimators):
    X_test_scaled = scaler.transform(X_test)
    X_test_scaled = pd.DataFrame(X_test_scaled, columns=X.columns, index=X_test.index)
    nn_predictions[:, i] = nn.predict(X_test_scaled)
    
y_pred_nn = np.mean(nn_predictions, axis=1)

# XGBoost
xgb_estimators = xgb_cv["estimator"]
xgb_predictions = np.zeros((X_test.shape[0], len(xgb_estimators)))

for i, xgb in enumerate(xgb_estimators):
    X_test_scaled = scaler.transform(X_test)
    X_test_scaled = pd.DataFrame(X_test_scaled, columns=X.columns, index=X_test.index)
    xgb_predictions[:, i] = xgb.predict(X_test_scaled)
    
y_pred_xgb = np.mean(xgb_predictions, axis=1)

In [ ]:
# compare the results


# Linear Regression
print(f"Linear Regression MSE: {mean_squared_error(y_test, y_pred_lr)}")

# Random Forest
print(f"Random Forest MSE: {mean_squared_error(y_test, y_pred_rf)}")

# Neural Network
print(f"Neural Network MSE: {mean_squared_error(y_test, y_pred_nn)}")

# XGBoost
print(f"XGBoost MSE: {mean_squared_error(y_test, y_pred_xgb)}")

fig, ax = plt.subplots(figsize=(10, 8))

# Plot Data
ax.plot(y_test.index, y_test, color="blue", label="Actual")
ax.plot(y_test.index, y_pred_lr, color="red", label="Linear Regression")
ax.plot(y_test.index, y_pred_rf, color="green", label="Random Forest")
ax.plot(y_test.index, y_pred_nn, color="purple", label="Neural Network")
ax.plot(y_test.index, y_pred_xgb, color="orange", label="XGBoost")

# Set labels and title
ax.set_xlabel("Date")
ax.set_ylabel("DOC (mg/l)")
ax.legend()

plt.show()

Note: complex models slightly outperform linear regression, but even better results can be achieved with hyperparameter tuning since these models have a lot of hyperparameters that can influence their performance. Also, we can perform different feature selection methods to avoid data-related problems such as multicollinearity.